### Lec 06-1: Softmax Regression (multinominal classification)

##### - 다음과 같이 3개의 구분이 필요한 경우, 앞서의 binominal이 3개 필요하다. 
#####    : 1) if A or not + 2) if B or not + 3) if C or not 
<left><img src= 'img/multi_classification.PNG' width="50%"></left>

##### - 따라서 3개의 행렬계산이 필요하며, 이를 3x3 행렬 하나로 표현할 수 있다. 
<left><img src= 'img/multi_classification2.PNG' width="25%"></left> <left><img src= 'img/multi_classification3.PNG' width="36.5%"></left>

### Lec 06-2: Cost function of Softmax Regression 

##### - 3개의 행렬연산 결과로 3개의 vector 값을 얻게 되는데, 여기에 softmax를 적용하여 확률로 변환한다. 
#####  : softmax는 입력값을 sigmoid와 같이 0~1의 값으로 변환하며, 동시에 변환된 값들의 합계가 1이 되도록 한다.
<left><img src= 'img/softmax.PNG' width="40%">

#####  - multinominal에서 사용하는 corss entropy cost 함수는, 앞서의 logistic cost 함수와 동일하다. 
<left><img src= 'img/cross_entropy1.PNG' width="40%"><left><img src= 'img/cross_entropy2.PNG' width="36%">
#####  - cost 함수는 아래와 같이 모형의 예측과 실제값간의 distance 평균으로 정의되며, Gradient Descent를 통해 이 값을 최소화 시키는 W를 찾아야 한다. 
<left><img src= 'img/cost_multi.PNG' width="30%">

### Lab 6-1 : Softmax Classification with tf

In [4]:
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
x_data = [[1,2,1,1], 
          [2,1,3,2], 
          [3,1,3,4], 
          [4,1,5,5], 
          [1,7,5,5], 
          [1,2,5,6], 
          [1,6,6,6], 
          [1,7,7,7]]

y_data = [[0,0,1], 
          [0,0,1], 
          [0,0,1], 
          [0,1,0], 
          [0,1,0], 
          [0,1,0], 
          [1,0,0],           
          [1,0,0]]

In [9]:
X = tf.placeholder('float', [None, 4])
Y = tf.placeholder('float', [None, 3])
nb_class = 3 
#### nb_class : number class, 여기서는 a/b/c의 3종류로 구분하는 문제이므로 클래스가 3임 

W = tf.Variable(tf.random_normal([4, nb_class]), name = 'wieght')
b = tf.Variable(tf.random_normal([nb_class]), name = 'bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

#### graphing 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
feed = {X:x_data, Y:y_data}
    
for step in range(2001): 
    sess.run( optimizer, feed_dict = feed  )
    if step % 200 == 0 : 
        print(step, sess.run(cost, feed_dict = feed) )

0 4.76265
200 0.65671456
400 0.5347403
600 0.43684354
800 0.34346002
1000 0.25807378
1200 0.22749545
1400 0.20679861
1600 0.18943605
1800 0.17466685
2000 0.16195798


In [21]:
#### 새로운 입력값에 대해서 결과를 예측해보자 
#### 계산값과 함께, argument max 명령어로 확률이 가장 높은 값의 위치(0, 1, 2)를 출력한다. 
new_data = [[1, 11, 7, 9]]

a = sess.run(hypothesis, feed_dict = {X:new_data})
print(a, sess.run(tf.arg_max(a, 1)))

[[6.9021583e-03 9.9309033e-01 7.5271387e-06]] [1]


In [22]:
new_data2 = [[1, 11, 7, 9], 
             [1, 3, 4, 3], 
             [1, 1, 0, 1] ]

b = sess.run(hypothesis, feed_dict = {X:new_data2})
print(b, sess.run(tf.arg_max(b, 1)))

[[6.9021713e-03 9.9309033e-01 7.5271528e-06]
 [8.4775984e-01 1.4439094e-01 7.8493282e-03]
 [1.6835154e-08 3.5959345e-04 9.9964035e-01]] [1 0 2]


### Lab 6-2 : Fancy Softmax Classification with tf

##### <font color = 'yellow'> - animal classification with softmax_cross_entropy_with_logits </font>
##### <font color = 'yellow'> - cross-entropy, one_hot, reshape 함수를 사용하여 코드를 간단하게 </font>

In [34]:
import numpy as np 
xy = np.loadtxt('data/data-04-zoo.txt', delimiter = ',', dtype = np.float32)
#xy

#### 동물들의 특징을 나타내는 컬럼들과, 맨 마지막 컬럼은 0~6으로 어떤 동물인지 구분 
x_data = xy[:, 0 :-1 ] 
y_data = xy[:, [-1] ] 

print(x_data.shape)
print(y_data.shape)

(101, 16)
(101, 1)


In [35]:
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])
nb_class = 7 


#### 0~6의 7개 값을 가진 Y를 onehot으로 recoding하고, 이를 다시 reshape하여 원하는 행렬을 만든다. 
#### onehot은 입력값 n+1개의 값을 만들기 때문에 reshape 과정이 필요하다. 
Y_one_hot = tf.one_hot(Y, nb_class)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_class])


W = tf.Variable(tf.random_normal([16, nb_class]), name = 'wieght')
b = tf.Variable(tf.random_normal([nb_class]), name = 'bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

#### cost 함수를 softmax_cross_entropy_with_logits를 사용하여 간단하게 ... 
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = tf.matmul(X, W) + b, labels = Y_one_hot)
cost = tf.reduce_mean(cost_i)

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

#### graphing 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
feed = {X:x_data, Y:y_data}
    
for step in range(2001): 
    sess.run( optimizer, feed_dict = feed  )
    if step % 200 == 0 : 
        print(step, sess.run(cost, feed_dict = feed) )

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0 6.016442
200 0.42918673
400 0.25095323
600 0.17637682
800 0.13502614
1000 0.10884621
1200 0.0909438
1400 0.078030154
1600 0.06832586
1800 0.060790006
2000 0.054778647


In [38]:
#### 모형의 정확도를 파악해보자 
prediction = tf.arg_max(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.arg_max(Y_one_hot, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for step in range(2001): 
    sess.run( optimizer, feed_dict = feed)
    if step % 100 == 0 : 
        loss, acc = sess.run([cost, accuracy], feed_dict = feed)
        print(step, loss, acc )

0 0.018991813 1.0
100 0.01869604 1.0
200 0.018409608 1.0
300 0.018132083 1.0
400 0.017863045 1.0
500 0.01760204 1.0
600 0.01734881 1.0
700 0.017102933 1.0
800 0.01686416 1.0
900 0.016632093 1.0
1000 0.016406525 1.0
1100 0.016187105 1.0
1200 0.015973663 1.0
1300 0.015765933 1.0
1400 0.015563659 1.0
1500 0.015366628 1.0
1600 0.015174667 1.0
1700 0.014987556 1.0
1800 0.014805144 1.0
1900 0.0146271745 1.0
2000 0.014453549 1.0


In [44]:
pred = sess.run(prediction, feed_dict = feed)

for p, y in zip(pred, y_data.flatten()): 
    print(p==int(y), "Prediction:", p, "True_Value:", int(y))

True Prediction: 0 True_Value: 0
True Prediction: 0 True_Value: 0
True Prediction: 3 True_Value: 3
True Prediction: 0 True_Value: 0
True Prediction: 0 True_Value: 0
True Prediction: 0 True_Value: 0
True Prediction: 0 True_Value: 0
True Prediction: 3 True_Value: 3
True Prediction: 3 True_Value: 3
True Prediction: 0 True_Value: 0
True Prediction: 0 True_Value: 0
True Prediction: 1 True_Value: 1
True Prediction: 3 True_Value: 3
True Prediction: 6 True_Value: 6
True Prediction: 6 True_Value: 6
True Prediction: 6 True_Value: 6
True Prediction: 1 True_Value: 1
True Prediction: 0 True_Value: 0
True Prediction: 3 True_Value: 3
True Prediction: 0 True_Value: 0
True Prediction: 1 True_Value: 1
True Prediction: 1 True_Value: 1
True Prediction: 0 True_Value: 0
True Prediction: 1 True_Value: 1
True Prediction: 5 True_Value: 5
True Prediction: 4 True_Value: 4
True Prediction: 4 True_Value: 4
True Prediction: 0 True_Value: 0
True Prediction: 0 True_Value: 0
True Prediction: 0 True_Value: 0
True Predi